In [7]:
import os
import sys
import re
import shutil
import time
import pickle
from collections import defaultdict
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas.plotting import register_matplotlib_converters
#register_matplotlib_converters()

In [86]:
import os
import re
import pandas as pd
import numpy as np

def fetch_data(data_dir):
    """
    Load all JSON formatted files from the specified directory into a DataFrame,
    ensuring correct column names and date formatting.
    """
    # Check if the directory exists and contains files
    if not os.path.isdir(data_dir):
        raise FileNotFoundError("Specified data directory does not exist")
    if not os.listdir(data_dir):
        raise ValueError("Specified data directory does not contain any files")

    # List of JSON files in the directory
    file_list = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.json')]

    # Expected columns and their possible alternatives
    correct_columns = ['country', 'customer_id', 'day', 'invoice', 'month', 'price', 'stream_id', 'times_viewed', 'year']
    column_map = {'StreamID': 'stream_id', 'TimesViewed': 'times_viewed', 'total_price': 'price'}

    # Read, clean, and concatenate JSON files
    all_months = []
    for file_name in file_list:
        df = pd.read_json(file_name)
        df.rename(columns={col: column_map.get(col, col) for col in df.columns}, inplace=True)
        
        # Validate columns
        if sorted(df.columns) != correct_columns:
            raise ValueError(f"Column names in {file_name} do not match the expected columns")

        all_months.append(df)

    # Concatenate all DataFrames
    df = pd.concat(all_months, ignore_index=True)
    
    # Construct invoice_date and clean invoice column
    df['invoice_date'] = pd.to_datetime(df[['year', 'month', 'day']])
    df['invoice'] = df['invoice'].astype(str).str.replace(r'\D+', '', regex=True)
    
    # Sort by date and reset index
    df.sort_values(by='invoice_date', inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df


In [87]:

# Define the data directory and fetch the data
data_dir = "./cs-train/"
df = fetch_data(data_dir)

# Save the DataFrame to a CSV file
output_file = "output_data.csv"
df.to_csv(output_file, index=False)


In [85]:
df.head()

,country,customer_id,day,invoice,month,price,stream_id,times_viewed,year,invoice_date
0,United Kingdom,13085.0,28,489434,11,6.95,85048,12,2017,2017-11-28
1,United Kingdom,13085.0,28,489434,11,6.75,79323W,12,2017,2017-11-28
2,United Kingdom,13085.0,28,489434,11,2.10,22041,21,2017,2017-11-28
3,United Kingdom,13085.0,28,489434,11,1.25,21232,5,2017,2017-11-28
4,United Kingdom,13085.0,28,489434,11,1.65,22064,17,2017,2017-11-28


In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 815011 entries, 0 to 815010
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   country       815011 non-null  object        
 1   customer_id   625249 non-null  float64       
 2   invoice       815011 non-null  object        
 3   price         815011 non-null  float64       
 4   stream_id     815011 non-null  object        
 5   times_viewed  815011 non-null  int64         
 6   year          815011 non-null  int64         
 7   month         815011 non-null  int64         
 8   day           815011 non-null  int64         
 9   invoice_date  815011 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(4), object(3)
memory usage: 62.2+ MB


In [76]:
df.isnull().sum()

country              0
customer_id     189762
invoice              0
price                0
stream_id            0
times_viewed         0
year                 0
month                0
day                  0
invoice_date         0
dtype: int64

In [77]:
df.describe().transpose()

,count,mean,min,25%,50%,75%,max,std
customer_id,625249.0,15333.415068,12346.0,13956.0,15279.0,16813.0,18287.0,1698.360788
price,815011.0,4.802631,-53594.36,1.25,2.1,4.21,38970.0,135.058707
times_viewed,815011.0,5.231106,0.0,1.0,3.0,8.0,24.0,5.109982
year,815011.0,2018.247654,2017.0,2018.0,2018.0,2019.0,2019.0,0.545261
month,815011.0,6.592718,1.0,3.0,6.0,10.0,12.0,3.598781
day,815011.0,15.064819,1.0,7.0,15.0,23.0,31.0,8.788845
invoice_date,815011,2018-10-01 18:14:32.866746112,2017-11-28 00:00:00,2018-05-16 00:00:00,2018-10-18 00:00:00,2019-02-06 00:00:00,2019-07-31 00:00:00,NaN


In [78]:
df.describe(include='object').transpose()

,count,unique,top,freq
country,815011,43,United Kingdom,751228
invoice,815011,42646,537434,1350
stream_id,815011,5007,85123A,5017


In [79]:
from datetime import datetime

# Define the min and max dates
min_date = datetime(2017, 11, 28)
max_date = datetime(2019, 7, 31)

# Calculate the difference in days
days_spanned = (max_date - min_date).days
print(f"The entire range of dates spans {days_spanned} days.")



The entire range of dates spans 610 days.


In [80]:
def country_with_max_revenue(df):
    """
    Identify the country with the highest total revenue.
    """
    # Group by 'country' and sum the 'price' for each group
    revenue_by_country = df.groupby('country')['price'].sum()
    
    # Find the country with the maximum revenue
    max_revenue_country = revenue_by_country.idxmax()
    max_revenue = revenue_by_country.max()
    
    return max_revenue_country, max_revenue

# Assuming df is the DataFrame returned from fetch_data function
max_revenue_country, max_revenue = country_with_max_revenue(df)
print(f"The country with the most total revenue is {max_revenue_country} with a total revenue of {max_revenue}.")


The country with the most total revenue is United Kingdom with a total revenue of 3521513.505.
